# Week 4: BBC News Classification Project
## DTSA 5510 - Intro to Machine Learning - Unsupervised

In [10]:
import pandas as pd
import numpy as np
import nltk
import string

from sklearn.feature_extraction.text import TfidfVectorizer


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bergs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bergs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bergs\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\bergs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### EDA & cleaning

The first step in any project is conducting some exploratory data analysis and cleaning.

EDA
* check for duplicate text
* check for missing text
* histogram of articles by category
  
Make pipleline to preprocess data
* remove missing text
* remove duplicate text
* tokenize text
* tfidf_vectorizer, plot scores

Make pipeline to Modeling
* non-negative matrix factorization models
    * Frobenius Loss
    * Kullback-Leibler Divergence 
    * plot top words per topic
* Compare to supervised learning model

In [2]:
# read in data
train_df = pd.read_csv('data/BBC News Train.csv')

train_df.head(5)

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.1+ KB


To visualize the occurence of each word in the articles, I'll first tokenize the text column (split the text into individual words).

In [8]:
# tokenize text column
train_df['tokens'] = train_df['Text'].apply(nltk.word_tokenize)

train_df.head(3)

,ArticleId,Text,Category,tokens
0,1833,worldcom ex-boss launches defence lawyers defe...,business,"[worldcom, ex-boss, launches, defence, lawyers..."
1,154,german business confidence slides german busin...,business,"[german, business, confidence, slides, german,..."
2,1101,bbc poll indicates economic gloom citizens in ...,business,"[bbc, poll, indicates, economic, gloom, citize..."


Next I'll remove the stop words like 'the', 'is', 'and' and so on. Stop words don't hold much significance in topic modelling and are unnecessary. None of the token words contain upper-case characters, so removal of those is unnecessary. 

In [11]:
nltk.download('stopwords')

# stop words
stop_words = set(stopwords.words('english'))
train_df['tokens'] = train_df['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# lowercase only
train_df['tokens'] = train_df['tokens'].apply(lambda x: [word.lower() for word in x])

# lemmatization
lemmatizer = WordNetLemmatizer()
train_df['tokens'] = train_df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Remove Punctuation
train_df['tokens'] = train_df['tokens'].apply(lambda x: [word for word in x if word.isalpha()])

# Join Tokens
train_df['processed_text'] = train_df['tokens'].apply(lambda x: ' '.join(x))

# Display the preprocessed text
train_df[['ArticleId', 'processed_text', 'Category']].head()

train_df.head(3)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bergs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ArticleId,Text,Category,tokens,processed_text
0,1833,worldcom ex-boss launches defence lawyers defe...,business,"[worldcom, launch, defence, lawyer, defending,...",worldcom launch defence lawyer defending forme...
1,154,german business confidence slides german busin...,business,"[german, business, confidence, slide, german, ...",german business confidence slide german busine...
2,1101,bbc poll indicates economic gloom citizens in ...,business,"[bbc, poll, indicates, economic, gloom, citize...",bbc poll indicates economic gloom citizen majo...


### Vectorization

In [ ]:
texts = train_df['processed_text']

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# Fit and transform the text data
dtm = vectorizer.fit_transform(texts)

In [ ]:
# Convert the sparse matrix to a dense format (only for inspection, as this can be memory-intensive)
dtm_dense = dtm.toarray()

# Get the feature names (terms)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame for easier inspection
dtm_df = pd.DataFrame(dtm_dense, columns=feature_names)

# Display the first few rows of the matrix
dtm_df.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sum the frequencies of each term across all documents
word_frequencies = np.sum(dtm.toarray(), axis=0)

# Create a DataFrame for the word frequencies
freq_df = pd.DataFrame({'Term': feature_names, 'Frequency': word_frequencies})

# Sort the DataFrame by frequency
freq_df = freq_df.sort_values(by='Frequency', ascending=False)

# Display the top 10 most frequent words
freq_df.head(10)


In [ ]:
# Plot the top 20 most frequent words
plt.figure(figsize=(10, 8))
sns.barplot(x='Frequency', y='Term', data=freq_df.head(20))
plt.title('Top 20 Most Frequent Words')
plt.xlabel('Frequency')
plt.ylabel('Words')
plt.show()


In [ ]:
from sklearn.decomposition import NMF

# Initialize the NMF model
num_topics = 5  # Choose the number of topics
nmf_model = NMF(n_components=num_topics, random_state=42)

# Fit the model to the document-term matrix
W = nmf_model.fit_transform(dtm)
H = nmf_model.components_


In [ ]:
# Get feature names (terms)
feature_names = vectorizer.get_feature_names_out()

# Print top terms for each topic
num_top_words = 10  # Number of top words to display for each topic
for topic_idx, topic in enumerate(H):
    top_terms_idx = topic.argsort()[-num_top_words:][::-1]
    top_terms = [feature_names[i] for i in top_terms_idx]
    print(f"Topic {topic_idx}:")
    print(" ".join(top_terms))
    print()


In [ ]:
# Assign the topic with the highest score to each document
train_df['Topic'] = W.argmax(axis=1)

# Display the first few rows with assigned topics
train_df[['ArticleId', 'Category', 'Topic']].head()


In [ ]:
# Plot the distribution of topics across documents
plt.figure(figsize=(10, 6))
sns.countplot(x='Topic', data=train_df)
plt.title('Distribution of Topics Across Documents')
plt.xlabel('Topic')
plt.ylabel('Number of Documents')
plt.show()
